In [1]:
import nba_py
import datetime
import time
import nba_py.game
import nba_py.player
import nba_py.team
import pandas as pd
import numpy as np
import functools
import pymysql
from sqlalchemy import create_engine
%matplotlib inline

In [2]:
conn = create_engine('mysql+pymysql://root:@localhost:3306/nba_stats')
game_logs = pd.read_sql_table('game_logs', conn)

In [7]:
game_logs.columns

Index(['SEASON_ID', 'Player_ID', 'Game_ID', 'GAME_DATE', 'MATCHUP', 'WL',
       'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
       'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'PTS', 'PLUS_MINUS', 'VIDEO_AVAILABLE', 'LOCATION', 'Oppo', 'Oppo_ID'],
      dtype='object')

In [10]:
game_log_col = ['SEASON_ID', 'Player_ID','LOCATION', 'Oppo_ID', 
                'MIN', 'PTS', 'AST', 'OREB', 'DREB', 'STL', 'BLK', 'TOV', 
                'FGM', 'FGA', 'FG3M', 'FG3A']

In [20]:
game_logs = game_logs[game_log_col]
game_logs = game_logs[game_logs['MIN'] != 0]

In [21]:
convert_to_36 = lambda x: x[['PTS', 'AST', 'OREB', 'DREB', 'STL', 'BLK', 
                             'TOV', 'FGM', 'FGA', 'FG3M']] * 36 / x['MIN']
game_logs_36 = game_logs.apply(convert_to_36, axis = 1)

In [29]:
game_logs_36 = pd.concat([game_logs[['Player_ID','LOCATION', 'Oppo_ID']], game_logs_36], axis = 1)

In [32]:
game_logs_36['SCORE'] = game_logs_36['PTS'] * 1 + game_logs_36['AST'] * 1.5 + \
                        game_logs_36['OREB'] * 1 + game_logs_36['DREB'] * 0.7 + \
                        game_logs_36['STL'] * 2 + game_logs_36['BLK'] * 1.8 + game_logs_36['TOV'] * -1 + \
                        game_logs_36['FGM'] * 0.4 + \
                        (game_logs_36['FGA'] - game_logs_36['FGM']) * -1 + game_logs_36['FG3M'] * 0.5
game_logs_36

,Player_ID,LOCATION,Oppo_ID,PTS,AST,OREB,DREB,STL,BLK,TOV,FGM,FGA,FG3M,SCORE
0,203518,HOME,1610612743,22.000000,2.000000,6.000000,6.000000,0.000000,0.000000,2.000000,8.000000,22.000000,6.000000,25.400000
1,203518,AWAY,1610612750,14.400000,0.000000,0.000000,0.000000,2.400000,2.400000,2.400000,4.800000,7.200000,4.800000,23.040000
2,203518,HOME,1610612749,36.000000,0.000000,0.000000,3.000000,0.000000,0.000000,3.000000,12.000000,18.000000,12.000000,39.900000
3,203518,HOME,1610612766,0.000000,0.000000,0.000000,5.538462,0.000000,0.000000,2.769231,0.000000,11.076923,0.000000,-9.969231
4,203518,HOME,1610612759,12.857143,2.571429,0.000000,0.000000,0.000000,0.000000,2.571429,5.142857,12.857143,2.571429,9.771429
5,203518,AWAY,1610612753,9.818182,1.636364,1.636364,0.000000,3.272727,0.000000,0.000000,3.272727,13.090909,3.272727,13.581818
6,203518,AWAY,1610612742,0.000000,0.000000,0.000000,10.285714,0.000000,0.000000,5.142857,0.000000,10.285714,0.000000,-8.228571
7,203518,AWAY,1610612745,14.666667,2.666667,0.000000,0.000000,4.000000,0.000000,1.333333,5.333333,13.333333,4.000000,21.466667
8,203518,HOME,1610612755,7.500000,0.000000,0.000000,4.500000,1.500000,1.500000,1.500000,1.500000,9.000000,1.500000,8.700000
9,203518,HOME,1610612744,3.600000,3.600000,0.000000,5.400000,3.600000,0.000000,0.000000,0.000000,14.400000,0.000000,5.580000


In [64]:
game_logs_36[(game_logs_36['Player_ID'] == 1626164) & (game_logs_36['LOCATION'] == 'HOME')]['SCORE'].describe()

count    79.000000
mean     21.739667
std      13.799493
min     -36.000000
25%      13.971619
50%      21.600000
75%      27.690000
max      70.200000
Name: SCORE, dtype: float64

In [65]:
game_logs_36[(game_logs_36['Player_ID'] == 1626164) & (game_logs_36['LOCATION'] == 'AWAY')]['SCORE'].describe()

count    75.000000
mean     19.863884
std      12.321743
min     -13.500000
25%      11.072727
50%      22.220690
75%      26.855212
max      63.520000
Name: SCORE, dtype: float64

In [66]:
game_logs_36[game_logs_36['Player_ID'] == 1626164]['SCORE'].describe()

count    154.000000
mean      20.826137
std       13.091852
min      -36.000000
25%       12.490435
50%       21.910345
75%       27.531000
max       70.200000
Name: SCORE, dtype: float64